<a href="https://colab.research.google.com/github/pachterlab/COVID-19-testing-database/blob/master/notebooks/basic_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --output-file="logs.csv" "https://docs.google.com/spreadsheets/d/10I8bVkLzvrmXJsb5N-8JSFpWw5vBwDKYzyOVAI4viKo/export?format=csv&gid=1514440859" -O "sheet.csv"

In [2]:
!awk 'NR>4' sheet.csv > data.csv

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

def nd(arr):
  return np.asarray(arr).reshape(-1)

def yex(ax):
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]
    
    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

In [4]:
df = pd.read_csv("data.csv")
df.index = df.institution.values
df["testing_num"] = df.testing.map(lambda x: {"FALSE": 0, "TRUE": 1}.get(x, 0))

In [5]:
df.head()

,institution,type,state,testing_info_link,rtc_model,last_updated,testing,testing_type,test_initial (True/False),test_recurring,test_freq (times/week),test_vendor,test_pool,comments,testing_num
Abilene Christian University,Abilene Christian University,Private,TX,https://www.acu.edu/coronavirus/july-17-2020-c...,Planning for in-person,7/22/2020,TRUE,Both asymptomatic and symptomatic,FALSE,FALSE,0.00,NaN,NaN,"Our plan includes testing any faculty, staff a...",1
Academy of Art University,Academy of Art University,Private,CA,https://www.academyart.edu/academy-of-art-univ...,Proposing a hybrid model,7/27/2020,Unknown,Unknown,Unknown,Unknown,NaN,NaN,NaN,NaN,0
Adelphi University,Adelphi University,Private,NY,https://www.adelphi.edu/restart/wp-content/upl...,Proposing a hybrid model,7/27/2020,TRUE,Symptomatic,TRUE,TRUE,0.33,Vault Health,NaN,"Re-entry testing of residential students, inte...",1
Adrian College,Adrian College,Private,MI,http://adrian.edu/news/ac-president-docking-sa...,Planning for in-person,7/27/2020,FALSE,No voluntary testing,FALSE,FALSE,0.00,NaN,NaN,Students are required to have a negative test ...,0
Agnes Scott College,Agnes Scott College,Private,GA,https://www.agnesscott.edu/coronavirus/updates...,Planning for in-person,7/22/2020,TRUE,Unknown,TRUE,Unknown,0.00,Unknown,NaN,"Athletes will be tested upon arrival, students...",1


In [6]:
df.columns

Index(['institution', 'type', 'state', 'testing_info_link', 'rtc_model',
       'last_updated', 'testing', 'testing_type', 'test_initial (True/False)',
       'test_recurring', 'test_freq (times/week)', 'test_vendor', 'test_pool',
       'comments', 'testing_num'],
      dtype='object')

In [7]:
df.groupby(["type", "test_initial (True/False)", "test_recurring"])["testing"].count().sort_values()


type     test_initial (True/False)  test_recurring
Private  Unknown                    TRUE                1
         FALSE                      Unknown             2
         Unknown                    FALSE               3
Public   FALSE                      TRUE                3
         TRUE                       Unknown             3
         FALSE                      Unknown             4
Private  FALSE                      TRUE                5
Public   TRUE                       FALSE               9
Private  TRUE                       Unknown            12
                                    FALSE              13
Public   TRUE                       TRUE               16
         Unknown                    Unknown            18
Private  Unknown                    Unknown            24
         TRUE                       TRUE               54
         FALSE                      FALSE             110
Public   FALSE                      FALSE             124
Name: testing, dtype:

In [9]:
pd.DataFrame(df.test_vendor.value_counts()).head(10)

,test_vendor
Broad Institute,16
Student Health,4
Vault Health,2
Quest,2
LabCorp,2
unlisted,2
In house lab,2
University Health Center,2
Student Health Services,2
University of Alabama at Birmingham,1
